# **PROTOTIPO FUNCIONAL DE DETECCIÓN DE VIOLENCIA FÍSICAS**

In [1]:
!pip install ultralytics deep-sort-realtime transformers torch opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [53]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


## PROCESAMIENTO DE VIDEOS A 1280x720

In [2]:
import os
from moviepy.editor import VideoFileClip

# PARÁMETROS PREDEFINIDOS - MODIFICA ESTOS VALORES
INPUT_PATH = "/content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba3.mp4"  # Ruta al video de entrada
OUTPUT_FOLDER = "/content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados"  # Carpeta donde se guardará el video procesado
START_TIME = 9  # Tiempo de inicio para el recorte (en segundos)
END_TIME = 14  # Tiempo de fin para el recorte (en segundos)
TARGET_RESOLUTION = (1280, 720)  # Resolución objetivo (ancho, alto)

def process_video(input_path, output_folder, start_time, end_time, target_resolution=(1280, 720)):
    """
    Procesa un video recortándolo por un rango de tiempo específico y ajustando su resolución.
    Mantiene el nombre original del archivo de entrada.

    Args:
        input_path (str): Ruta al video de entrada
        output_folder (str): Carpeta donde se guardará el video procesado
        start_time (float): Tiempo de inicio para el recorte (en segundos)
        end_time (float): Tiempo de fin para el recorte (en segundos)
        target_resolution (tuple): Resolución objetivo en formato (ancho, alto)

    Returns:
        str: Ruta del archivo de salida procesado
    """
    # Verificar si el archivo de entrada existe
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de video no existe: {input_path}")

    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Carpeta de salida creada: {output_folder}")

    # Obtener el nombre base del archivo de entrada sin extensión
    base_name = os.path.basename(input_path)
    file_name, file_ext = os.path.splitext(base_name)

    # Mantener el nombre original del archivo
    output_name = f"{file_name}{file_ext}"
    output_path = os.path.join(output_folder, output_name)

    try:
        # Cargar el video
        print(f"Cargando video: {input_path}")
        video = VideoFileClip(input_path)

        # Obtener información del video original
        original_duration = video.duration
        original_size = video.size
        print(f"Información del video original:")
        print(f"- Duración: {original_duration:.2f} segundos")
        print(f"- Resolución: {original_size[0]}x{original_size[1]}")

        # Validar tiempos de recorte
        if start_time < 0 or end_time > original_duration or start_time >= end_time:
            raise ValueError(f"Tiempos de recorte inválidos: {start_time}s - {end_time}s. " +
                            f"El video tiene una duración de {original_duration:.2f}s")

        # Recortar el video dentro del rango especificado
        print(f"Recortando video del segundo {start_time} al {end_time}")
        video_trimmed = video.subclip(start_time, end_time)

        # Redimensionar el video si es necesario
        if original_size[0] != target_resolution[0] or original_size[1] != target_resolution[1]:
            print(f"Redimensionando video de {original_size[0]}x{original_size[1]} a {target_resolution[0]}x{target_resolution[1]}")
            # Usar redimensionamiento compatible con moviepy 1.0.3 y Pillow 6.2.2
            video_processed = video_trimmed.resize(target_resolution)
        else:
            print("El video ya tiene la resolución objetivo. No es necesario redimensionar.")
            video_processed = video_trimmed

        # Guardar el video procesado con alta calidad
        print(f"Guardando video procesado: {output_path}")

        # Configurar parámetros de codificación para mantener buena calidad
        video_processed.write_videofile(
            output_path,
            codec="libx264",
            audio_codec="aac",
            bitrate="8000k",
            preset="slow",  # Mejor calidad
            threads=4,
            fps=video.fps,  # Mantener la misma tasa de fotogramas
            temp_audiofile="temp-audio.m4a",
            remove_temp=True
        )

        # Obtener información del video procesado
        print(f"Video procesado guardado exitosamente:")
        print(f"- Duración: {end_time - start_time:.2f} segundos")
        print(f"- Resolución: {target_resolution[0]}x{target_resolution[1]}")

        # Limpiar
        video.close()
        video_processed.close()

        return output_path

    except Exception as e:
        print(f"Error al procesar el video: {str(e)}")
        # Asegurar que los recursos se liberen en caso de error
        try:
            video.close()
        except:
            pass
        try:
            video_processed.close()
        except:
            pass
        raise

# Punto de entrada principal del script
if __name__ == "__main__":
    print("=" * 50)
    print("PROCESADOR DE VIDEO: RECORTE Y REDIMENSIONAMIENTO")
    print("=" * 50)
    print(f"Video de entrada: {INPUT_PATH}")
    print(f"Carpeta de salida: {OUTPUT_FOLDER}")
    print(f"Rango de recorte: {START_TIME}s - {END_TIME}s")
    print(f"Resolución objetivo: {TARGET_RESOLUTION[0]}x{TARGET_RESOLUTION[1]}")
    print("=" * 50)

    try:
        # Ejecutar procesamiento con los parámetros predefinidos
        result = process_video(INPUT_PATH, OUTPUT_FOLDER, START_TIME, END_TIME, TARGET_RESOLUTION)
        print("\n" + "=" * 50)
        print(f"PROCESO COMPLETADO EXITOSAMENTE")
        print(f"Video procesado guardado en: {result}")
        print("=" * 50)
    except Exception as e:
        print("\n" + "=" * 50)
        print(f"ERROR: {str(e)}")
        print("=" * 50)

  if event.key is 'enter':



PROCESADOR DE VIDEO: RECORTE Y REDIMENSIONAMIENTO
Video de entrada: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba3.mp4
Carpeta de salida: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados
Rango de recorte: 9s - 14s
Resolución objetivo: 1280x720
Cargando video: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba3.mp4
Información del video original:
- Duración: 5.00 segundos
- Resolución: 768x432
Error al procesar el video: Tiempos de recorte inválidos: 9s - 14s. El video tiene una duración de 5.00s

ERROR: Tiempos de recorte inválidos: 9s - 14s. El video tiene una duración de 5.00s


In [55]:
import os
from moviepy.editor import VideoFileClip

# PARÁMETROS PREDEFINIDOS - MODIFICA ESTOS VALORES
INPUT_PATH = "/content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba10.mp4"  # Ruta al video de entrada
OUTPUT_FOLDER = "/content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados"  # Carpeta donde se guardará el video procesado
TARGET_RESOLUTION = (1280, 720)  # Resolución objetivo (ancho, alto)

def process_video(input_path, output_folder, target_resolution=(1280, 720)):
    """
    Procesa un video ajustando su resolución.
    Mantiene el nombre original del archivo de entrada.

    Args:
        input_path (str): Ruta al video de entrada
        output_folder (str): Carpeta donde se guardará el video procesado
        target_resolution (tuple): Resolución objetivo en formato (ancho, alto)

    Returns:
        str: Ruta del archivo de salida procesado
    """
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de video no existe: {input_path}")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Carpeta de salida creada: {output_folder}")

    base_name = os.path.basename(input_path)
    file_name, file_ext = os.path.splitext(base_name)
    output_name = f"{file_name}{file_ext}"
    output_path = os.path.join(output_folder, output_name)

    try:
        print(f"Cargando video: {input_path}")
        video = VideoFileClip(input_path)

        original_size = video.size
        original_duration = video.duration
        print(f"Información del video original:")
        print(f"- Duración: {original_duration:.2f} segundos")
        print(f"- Resolución: {original_size[0]}x{original_size[1]}")

        if original_size != list(target_resolution):
            print(f"Redimensionando video a {target_resolution[0]}x{target_resolution[1]}")
            video_processed = video.resize(target_resolution)
        else:
            print("El video ya tiene la resolución objetivo. No es necesario redimensionar.")
            video_processed = video

        print(f"Guardando video procesado: {output_path}")
        video_processed.write_videofile(
            output_path,
            codec="libx264",
            audio_codec="aac",
            bitrate="8000k",
            preset="slow",
            threads=4,
            fps=video.fps,
            temp_audiofile="temp-audio.m4a",
            remove_temp=True
        )

        print(f"Video procesado guardado exitosamente.")
        print(f"- Duración: {original_duration:.2f} segundos")
        print(f"- Resolución: {target_resolution[0]}x{target_resolution[1]}")

        video.close()
        video_processed.close()

        return output_path

    except Exception as e:
        print(f"Error al procesar el video: {str(e)}")
        try:
            video.close()
        except:
            pass
        try:
            video_processed.close()
        except:
            pass
        raise

# Punto de entrada principal del script
if __name__ == "__main__":
    print("=" * 50)
    print("PROCESADOR DE VIDEO: SOLO REDIMENSIONAMIENTO")
    print("=" * 50)
    print(f"Video de entrada: {INPUT_PATH}")
    print(f"Carpeta de salida: {OUTPUT_FOLDER}")
    print(f"Resolución objetivo: {TARGET_RESOLUTION[0]}x{TARGET_RESOLUTION[1]}")
    print("=" * 50)

    try:
        result = process_video(INPUT_PATH, OUTPUT_FOLDER, TARGET_RESOLUTION)
        print("\n" + "=" * 50)
        print("PROCESO COMPLETADO EXITOSAMENTE")
        print(f"Video procesado guardado en: {result}")
        print("=" * 50)
    except Exception as e:
        print("\n" + "=" * 50)
        print(f"ERROR: {str(e)}")
        print("=" * 50)


PROCESADOR DE VIDEO: SOLO REDIMENSIONAMIENTO
Video de entrada: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba10.mp4
Carpeta de salida: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados
Resolución objetivo: 1280x720
Cargando video: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba10.mp4
Información del video original:
- Duración: 5.00 segundos
- Resolución: 1280x720
El video ya tiene la resolución objetivo. No es necesario redimensionar.
Guardando video procesado: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados/video_prueba10.mp4
Moviepy - Building video /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados/video_prueba10.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados/video_prueba10.mp4



t:  99%|█████████▉| 150/151 [00:15<00:00,  5.43it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_nuevos/video_prueba10.mp4, 2764800 bytes wanted but 0 bytes read,at frame 150/151, at time 5.00/5.00 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados/video_prueba10.mp4
Video procesado guardado exitosamente.
- Duración: 5.00 segundos
- Resolución: 1280x720

PROCESO COMPLETADO EXITOSAMENTE
Video procesado guardado en: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados/video_prueba10.mp4


In [ ]:
!pip install ultralytics deep-sort-realtime transformers torch opencv-python numpy

In [3]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from transformers import TimesformerForVideoClassification, AutoImageProcessor
import logging
import os
from collections import deque, defaultdict
from datetime import datetime

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [18]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("violence_detection_prototype.log"),
        logging.StreamHandler()
    ]
)

# Definir rutas
BASE_PATH = "/content/drive/MyDrive/Proyecto IA-3/violence_school_project/"
YOLO_MODEL_PATH = os.path.join(BASE_PATH, "models/yolo_V2/save_models/finetune/weights/best.pt")  # Modelo YOLO entrenado
TIMESFORMER_MODEL_PATH = os.path.join(BASE_PATH, "models/timesformer/run_finetune_20250408_003512/best_timesformer_finetune.pt")
# TIMESFORMER_MODEL_PATH = os.path.join(BASE_PATH, "models/timesformer/run_20250407_115035/best_timesformer_transfer.pt")
VIDEO_PATH = os.path.join(BASE_PATH, "videos/videos_procesados/video_prueba10.mp4")  # Video de prueba
OUTPUT_PATH = os.path.join(BASE_PATH, "output_videos/video_prueba10.mp4")

# Verificar que los modelos existan
if not os.path.exists(YOLO_MODEL_PATH):
    raise FileNotFoundError(f"No se encontró el modelo YOLO en: {YOLO_MODEL_PATH}")
if not os.path.exists(TIMESFORMER_MODEL_PATH):
    raise FileNotFoundError(f"No se encontró el modelo TimeSformer en: {TIMESFORMER_MODEL_PATH}")

# Crear la carpeta de salida si no existe
output_dir = os.path.dirname(OUTPUT_PATH)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    logging.info(f"Carpeta de salida creada: {output_dir}")

In [5]:
# Configurar dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Usando dispositivo: {device}")
print(f"Usando dispositivo: {device}")

# Cargar modelo YOLO entrenado
yolo_model = YOLO(YOLO_MODEL_PATH)
logging.info(f"Modelo YOLO cargado desde: {YOLO_MODEL_PATH}")
print(f"Modelo YOLO cargado desde: {YOLO_MODEL_PATH}")

# Cargar DeepSORT
deepsort = DeepSort(max_age=30, n_init=3, nn_budget=100)
logging.info("DeepSORT inicializado.")
print("DeepSORT inicializado.")

# Cargar modelo TimeSformer
timesformer_model = TimesformerForVideoClassification.from_pretrained(TIMESFORMER_MODEL_PATH)
timesformer_model.to(device)
timesformer_model.eval()
processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
logging.info(f"Modelo TimeSformer cargado desde: {TIMESFORMER_MODEL_PATH}")
print(f"Modelo TimeSformer cargado desde: {TIMESFORMER_MODEL_PATH}")

Usando dispositivo: cuda
Modelo YOLO cargado desde: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/models/yolo_V2/save_models/finetune/weights/best.pt
DeepSORT inicializado.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Modelo TimeSformer cargado desde: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/models/timesformer/run_finetune_20250408_003512/best_timesformer_finetune.pt


In [61]:
# Parámetros
CLIP_DURATION_SECONDS = 10  # Aumentar a 10 segundos para más contexto
FPS = 30  # FPS del video de entrada (valor inicial, se ajustará)
CLIP_FRAMES = CLIP_DURATION_SECONDS * FPS
STRIDE_FRAMES = CLIP_FRAMES // 5  # Procesar cada 2 segundos (más solapamiento)
TIMESFORMER_FPS = 15
TIMESFORMER_FRAMES = CLIP_DURATION_SECONDS * TIMESFORMER_FPS
NUM_FRAMES_TIMESFORMER = 8
THRESHOLD_VIOLENCE = 0.6  # Aumentar el umbral para reducir falsos positivos
YOLO_CONF_THRESHOLD = 0.65  # Aumentar para reducir detecciones duplicadas

# Cola para almacenar frames para TimeSformer
frame_buffer = deque(maxlen=CLIP_FRAMES)

# Estructura para almacenar trayectorias de DeepSORT por frame
trajectories = defaultdict(list)

# Función para preprocesar frames para TimeSformer
def preprocess_frames_for_timesformer(frames, num_frames=NUM_FRAMES_TIMESFORMER, target_size=(224, 224), target_fps=TIMESFORMER_FPS):
    # Reducir FPS a 15 FPS
    total_frames = len(frames)
    target_frame_count = int((total_frames / FPS) * target_fps)
    frame_indices = np.linspace(0, total_frames - 1, target_frame_count, dtype=int)
    selected_frames = [frames[i] for i in frame_indices]

    # Muestrear 8 frames uniformemente
    if len(selected_frames) < num_frames:
        raise ValueError(f"No hay suficientes frames después de reducir FPS: {len(selected_frames)} < {num_frames}")

    sample_indices = np.linspace(0, len(selected_frames) - 1, num_frames, dtype=int)
    final_frames = [selected_frames[i] for i in sample_indices]

    # Redimensionar y convertir a RGB
    processed_frames = []
    for frame in final_frames:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w = frame.shape[:2]
        ratio = min(target_size[0] / w, target_size[1] / h)
        new_w, new_h = int(w * ratio), int(h * ratio)
        resized_frame = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)
        padded_frame = np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
        pad_top = (target_size[1] - new_h) // 2
        pad_left = (target_size[0] - new_w) // 2
        padded_frame[pad_top:pad_top + new_h, pad_left:pad_left + new_w] = resized_frame
        processed_frames.append(padded_frame)

    # Preprocesar con AutoImageProcessor
    inputs = processor(processed_frames, return_tensors="pt")
    pixel_values = inputs["pixel_values"].to(device)
    return pixel_values

# Función para predecir violencia con TimeSformer
def predict_violence(frames):
    try:
        pixel_values = preprocess_frames_for_timesformer(frames)
        with torch.no_grad():
            outputs = timesformer_model(pixel_values=pixel_values)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)
            prob_violence = probs[0, 1].item()
            pred = 1 if prob_violence > THRESHOLD_VIOLENCE else 0
        return pred, prob_violence
    except Exception as e:
        logging.error(f"Error al predecir violencia: {str(e)}")
        return 0, 0.0

# Función para obtener los IDs presentes en un intervalo de frames
def get_ids_in_interval(start_frame, end_frame):
    ids_in_interval = set()
    for frame_num in range(start_frame, end_frame + 1):
        if frame_num in trajectories:
            for track_id, _ in trajectories[frame_num]:
                ids_in_interval.add(track_id)
    return sorted(list(ids_in_interval))


In [62]:
# Cargar video
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise ValueError(f"No se pudo abrir el video: {VIDEO_PATH}")

# Obtener propiedades del video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Ajustar FPS si es necesario
if fps != FPS:
    logging.warning(f"El FPS del video ({fps}) no coincide con el esperado ({FPS}). Ajustando...")
    print(f"El FPS del video ({fps}) no coincide con el esperado ({FPS}). Ajustando...")
    FPS = fps
    CLIP_FRAMES = CLIP_DURATION_SECONDS * FPS
    STRIDE_FRAMES = CLIP_FRAMES // 5
    TIMESFORMER_FRAMES = CLIP_DURATION_SECONDS * TIMESFORMER_FPS
    frame_buffer = deque(maxlen=CLIP_FRAMES)

# Configurar video de salida
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, FPS, (width, height))

# Verificar que VideoWriter se inicializó correctamente
if not out.isOpened():
    raise ValueError(f"No se pudo inicializar el VideoWriter para: {OUTPUT_PATH}. Verifica permisos y códec.")

frame_count = 0
written_frames = 0
violence_detected = False
last_prob_violence = 0.0
last_violence_ids = []

logging.info(f"Procesando video: {VIDEO_PATH}")
print(f"Procesando video: {VIDEO_PATH}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Redimensionar frame para YOLO
    yolo_frame = cv2.resize(frame, (640, 640), interpolation=cv2.INTER_AREA)
    frame_buffer.append(frame.copy())

    # Detección con YOLO
    results = yolo_model(yolo_frame, conf=YOLO_CONF_THRESHOLD, classes=0, iou=0.5)
    detections = []
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        scale_x = width / 640
        scale_y = height / 640
        for box, score in zip(boxes, scores):
            x1, y1, x2, y2 = box
            x1, x2 = x1 * scale_x, x2 * scale_x
            y1, y2 = y1 * scale_y, y2 * scale_y
            detections.append(([x1, y1, x2 - x1, y2 - y1], score, 0))

    # Seguimiento con DeepSORT
    tracks = deepsort.update_tracks(detections, frame=frame)

    # Almacenar trayectorias
    current_trajectories = []
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        current_trajectories.append((track_id, ltrb))
    trajectories[frame_count] = current_trajectories

    # Predecir violencia cada STRIDE_FRAMES
    if frame_count % STRIDE_FRAMES == 0 and len(frame_buffer) == CLIP_FRAMES:
        start_frame = max(1, frame_count - CLIP_FRAMES + 1)
        end_frame = frame_count

        pred, prob_violence = predict_violence(list(frame_buffer))
        # Post-procesamiento: si la probabilidad está cerca del umbral, no clasificar como violencia
        if 0.4 < prob_violence < 0.6:
            pred = 0  # Considerar como "incierto"
        violence_detected = (pred == 1)
        last_prob_violence = prob_violence

        logging.info(f"Predicción en frames {start_frame}-{end_frame}: Violencia={'Sí' if violence_detected else 'No'}, Probabilidad={prob_violence:.4f}")
        print(f"Predicción en frames {start_frame}-{end_frame}: Violencia={'Sí' if violence_detected else 'No'}, Probabilidad={prob_violence:.4f}")

        ids_in_interval = get_ids_in_interval(start_frame, end_frame)
        last_violence_ids = ids_in_interval if violence_detected else []

        if violence_detected:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            event = f"Violencia detectada en frames {start_frame}-{end_frame}, Probabilidad: {prob_violence:.4f}, IDs: {ids_in_interval}, Timestamp: {timestamp}"
            logging.info(event)
            print(event)

    # Visualización
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar predicción de violencia
    status_text = f"Violencia: {'Sí' if violence_detected else 'No'} (Prob: {last_prob_violence:.4f})"
    status_color = (0, 0, 255) if violence_detected else (0, 255, 0)
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2)

    # Mostrar IDs involucrados en violencia
    if violence_detected and last_violence_ids:
        ids_text = f"IDs Involucrados: {last_violence_ids}"
        cv2.putText(frame, ids_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Escribir frame en el video de salida
    out.write(frame)
    written_frames += 1

    # Mostrar progreso
    if frame_count % 100 == 0:
        logging.info(f"Procesados {frame_count}/{total_frames} frames")
        print(f"Procesados {frame_count}/{total_frames} frames")

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

Procesando video: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/videos/videos_procesados/video_prueba10.mp4

0: 640x640 2 personas, 14.7ms
Speed: 6.3ms preprocess, 14.7ms inference, 10.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 personas, 9.9ms
Speed: 4.9ms preprocess, 9.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 persona, 14.0ms
Speed: 6.2ms preprocess, 14.0ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 persona, 10.1ms
Speed: 2.9ms preprocess, 10.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 persona, 9.7ms
Speed: 2.9ms preprocess, 9.7ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 persona, 10.4ms
Speed: 3.1ms preprocess, 10.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 personas, 10.4ms
Speed: 3.1ms preprocess, 10.4ms inference, 2.0ms postprocess per image at shape (1, 3

In [63]:
# Verificar que el video de salida existe y reportar frames escritos
if os.path.exists(OUTPUT_PATH):
    logging.info(f"Video procesado guardado en: {OUTPUT_PATH}")
    print(f"Video procesado guardado en: {OUTPUT_PATH}")
    logging.info(f"Total de frames escritos: {written_frames}")
    print(f"Total de frames escritos: {written_frames}")
else:
    logging.error(f"No se encontró el video de salida en: {OUTPUT_PATH}. Verifica permisos y códec.")
    print(f"No se encontró el video de salida en: {OUTPUT_PATH}. Verifica permisos y códec.")

Video procesado guardado en: /content/drive/MyDrive/Proyecto IA-3/violence_school_project/output_videos/video_prueba10.mp4
Total de frames escritos: 151
